In [5]:
import java.io.File
import java.net.HttpURLConnection
import java.net.URI

// Step 1: Create folders
val currentDir = System.getProperty("user.dir")

val doomRPGFolder = File(currentDir, "DoomRPG")
doomRPGFolder.mkdirs()

val gameFolder = File(doomRPGFolder, "Game")
gameFolder.mkdirs()
val tempFolder = File(doomRPGFolder, "Temporary")
tempFolder.mkdirs()

val filesFolder = File(tempFolder, "Files")
filesFolder.mkdirs()
val brewBinaryFolder = File(tempFolder, "Brew Binary")
brewBinaryFolder.mkdirs()
val desktopBinaryFolder = File(tempFolder, "Desktop Binary")
desktopBinaryFolder.mkdirs()

// Step 2: Download necessary files

val brewBinaryUrl = "https://archive.org/download/doomrpg_brew/doomrpg.zip"
val brewBinary = File(filesFolder, "doomrpg.zip")

println("Downloading Brew binary...")
URI(brewBinaryUrl).toURL().openStream().use { input ->
    brewBinary.outputStream().use { output ->
        input.copyTo(output)
    }
}
println("Brew binary downloaded → $brewBinary")

// Desktop binary (latest release source code zip from GitHub)
val desktopBinaryLatestReleasesPath = "https://github.com/Erick194/DoomRPG-RE/releases/latest"

println("Finding latest release from GitHub...")
val connection = URI(desktopBinaryLatestReleasesPath).toURL().openConnection() as HttpURLConnection
connection.instanceFollowRedirects = false
connection.connect()

val redirectUrl = connection.getHeaderField("Location") ?: desktopBinaryLatestReleasesPath
connection.disconnect()

val versionTag = redirectUrl.substringAfterLast("/")
val desktopBinaryUrl = "https://github.com/Erick194/DoomRPG-RE/archive/refs/tags/$versionTag.zip"
val desktopBinary = File(filesFolder, "DoomRPG-RE-$versionTag.zip")

println("Downloading Desktop binary (latest release source code zip) from $desktopBinaryUrl...")
try {
    URI(desktopBinaryUrl).toURL().openStream().use { input ->
        desktopBinary.outputStream().use { output ->
            input.copyTo(output)
        }
    }
    println("Desktop binary downloaded → $desktopBinary")
} catch (exception: Exception) {
    println("Error downloading: ${exception.message}")
    println("Please check the URL: $desktopBinaryUrl")
    println("You may need to download manually from the GitHub releases page.")
}

// ────────────────────────────────────────────────
// Variables for next steps:
// brew_binary: File     → Temporary/Files/doomrpg.zip
// desktop_binary: File  → Temporary/Files/DoomRPG-RE-<tag>.zip
// ────────────────────────────────────────────────

Desktop binary downloaded → C:\MikhailProgramms\Programming\Projects\Pet Projects\DoomRPG-RE\scripts\prepare_files_and_run_game\DoomRPG\Temporary\Files\DoomRPG-RE-v.0.2.2.zip
